In [6]:
import pandas as pd

all_df = pd.read_pickle('../../arxiv-metadata-oai-2019.pkl')
all_df

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,group
0,0704.0297,Sung-Chul Yoon,"Sung-Chul Yoon, Philipp Podsiadlowski and Step...",Remnant evolution after a carbon-oxygen white ...,"15 pages, 15 figures, 3 tables, submitted to M...",None,10.1111/j.1365-2966.2007.12161.x,None,astro-ph,None,We systematically explore the evolution of t...,"[{'version': 'v1', 'created': 'Tue, 3 Apr 2007...",2019-08-19,"[[Yoon, Sung-Chul, ], [Podsiadlowski, Philipp,...",Physics
1,0704.0342,Patrice Ntumba Pungu,B. Dugmore and PP. Ntumba,Cofibrations in the Category of Frolicher Spac...,27 pages,None,None,None,math.AT,None,Cofibrations are defined in the category of ...,"[{'version': 'v1', 'created': 'Tue, 3 Apr 2007...",2019-08-19,"[[Dugmore, B., ], [Ntumba, PP., ]]",Mathematics
2,0704.0360,Zaqarashvili,T.V. Zaqarashvili and K Murawski,Torsional oscillations of longitudinally inhom...,"6 pages, 3 figures, accepted in A&A",None,10.1051/0004-6361:20077246,None,astro-ph,None,We explore the effect of an inhomogeneous ma...,"[{'version': 'v1', 'created': 'Tue, 3 Apr 2007...",2019-08-19,"[[Zaqarashvili, T. V., ], [Murawski, K, ]]",Physics
3,0704.0525,"Sezgin Ayg\""un","Sezgin Aygun, Ismail Tarhan, Husnu Baysal",On the Energy-Momentum Problem in Static Einst...,This submission has been withdrawn by arXiv ad...,"Chin.Phys.Lett.24:355-358,2007",10.1088/0256-307X/24/2/015,None,gr-qc,None,This paper has been removed by arXiv adminis...,"[{'version': 'v1', 'created': 'Wed, 4 Apr 2007...",2019-10-21,"[[Aygun, Sezgin, ], [Tarhan, Ismail, ], [Baysa...",Physics
4,0704.0535,Antonio Pipino,"Antonio Pipino (1,3), Thomas H. Puzia (2,4), a...",The Formation of Globular Cluster Systems in M...,"32 pages (referee format), 9 figures, ApJ acce...","Astrophys.J.665:295-305,2007",10.1086/519546,None,astro-ph,None,The most massive elliptical galaxies show a ...,"[{'version': 'v1', 'created': 'Wed, 4 Apr 2007...",2019-08-19,"[[Pipino, Antonio, ], [Puzia, Thomas H., ], [M...",Physics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170613,quant-ph/9904032,Mikhail Lukin,"V. A. Sautenkov, M. D. Lukin, C. J. Bednar, G....",Enhancement of Magneto-Optic Effects via Large...,None,None,10.1103/PhysRevA.62.023810,None,quant-ph,None,We utilize the generation of large atomic co...,"[{'version': 'v1', 'created': 'Thu, 8 Apr 1999...",2019-08-17,"[[Sautenkov, V. A., ], [Lukin, M. D., ], [Bedn...",Physics
170614,solv-int/9511005,Wen-Xiu Ma,"Wen-Xiu Ma, Benno Fuchssteiner",Explicit and Exact Solutions to a Kolmogorov-P...,"14pages, Latex, to appear in Intern. J. Nonlin...",None,10.1016/0020-7462(95)00064-X,None,solv-int nlin.SI,None,Some explicit traveling wave solutions to a ...,"[{'version': 'v1', 'created': 'Tue, 14 Nov 199...",2019-08-15,"[[Ma, Wen-Xiu, ], [Fuchssteiner, Benno, ]]",Physics
170615,solv-int/9809008,Victor Enolskii,"J C Eilbeck, V Z Enol'skii, V B Kuznetsov, D V...",Linear r-Matrix Algebra for a Hierarchy of One...,"plain LaTeX, 28 pages",None,None,None,solv-int nlin.SI,None,We consider a hierarchy of many-particle sys...,"[{'version': 'v1', 'created': 'Wed, 2 Sep 1998...",2019-08-17,"[[Eilbeck, J C, ], [Enol'skii, V Z, ], [Kuznet...",Physics
170616,solv-int/9909010,Pierre van Moerbeke,"M. Adler, T. Shiota and P. van Moerbeke",Pfaff tau-functions,42 pages,None,None,None,solv-int adap-org hep-th nlin.AO nlin.SI,None,Consider the evolution $$ \frac{\pl m_\iy}{\...,"[{'version': 'v1', 'created': 'Wed, 15 Sep 199...",2019-08-17,"[[Adler, M., ], [Shiota, T., ], [van Moerbeke,...",Physics


# 8、获取论文页数、图、表信息

In [7]:
import re


def queryBycategrot(data, categrot):
    return data[data['group'].str.contains(categrot)]

# 生成描述信息
def get_description(df, opt):
    group_name = ['Computer Science', 'Economics', 'Electrical Engineering and Systems Science',
                  'Mathematics', 'Physics', 'Quantitative Biology', 'Quantitative Finance', 'Statistics']
    page_descriptions = {'Group': [], 'Len': [], 'Count': [], 'Percentage': [], 'Mean': [], 'Std': [], 'Min': [],
                         '25%': [],
                         '50%': [], '75%': [], 'Max': []}
    for i in group_name:
        gr_comments_df = queryBycategrot(df, i)
        re_str = '[1-9][0-9]* *' + opt
        gr_comments_df['pages'] = gr_comments_df['comments'].map(lambda x: re.findall(re_str, str(x)))
        # data_1 = gr_comments_df.copy()
        # data_1 = data_1[data_1['pages'].apply(len) > 0]
        ownpages_df = gr_comments_df.copy()
        ownpages_df = ownpages_df[ownpages_df['pages'].apply(len) > 0]
        ownpages_df['pages'] = ownpages_df['pages'].apply(lambda x: float(x[0].replace(opt, '')))
        gr_desc = ownpages_df['pages'].describe().astype(int)
        page_descriptions['Group'].append(i)
        page_descriptions['Len'].append(gr_comments_df.shape[0])
        page_descriptions['Count'].append(gr_desc['count'])
        page_descriptions['Percentage'].append(int((gr_desc['count'] / gr_comments_df.shape[0]) * 100) / 100)
        page_descriptions['Mean'].append(gr_desc['mean'])
        page_descriptions['Std'].append(gr_desc['std'])
        page_descriptions['Min'].append(gr_desc['min'])
        page_descriptions['25%'].append(gr_desc['25%'])
        page_descriptions['50%'].append(gr_desc['50%'])
        page_descriptions['75%'].append(gr_desc['75%'])
        page_descriptions['Max'].append(gr_desc['max'])
        page_descriptions['Mean'] = page_descriptions['Mean']
    return pd.DataFrame(page_descriptions)


for i in ['pages', 'tables', 'figures']:
    desc_df = get_description(all_df, i)
    desc_df.to_csv(f'../output/{i}-descriptions.csv', index=False)

C:\Users\马\AppData\Local\Temp\ipykernel_15124\179824035.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gr_comments_df['pages'] = gr_comments_df['comments'].map(lambda x: re.findall(re_str, str(x)))
C:\Users\马\AppData\Local\Temp\ipykernel_15124\179824035.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gr_comments_df['pages'] = gr_comments_df['comments'].map(lambda x: re.findall(re_str, str(x)))
C:\Users\马\AppData\Local\Temp\ipykernel_15124\179824035.py:17: SettingWithCopyWarning: 
A value is tryin

In [8]:
import re


def queryBycategrot(data, categrot):
    return data[data['group'].str.contains(categrot)]

# 生成具体数据
def get_datasofcommits(df, opt):
    group_name = ['Computer Science', 'Economics', 'Electrical Engineering and Systems Science',
                  'Mathematics', 'Physics', 'Quantitative Biology', 'Quantitative Finance', 'Statistics']
    page_data = pd.DataFrame()
    for i in group_name:
        gr_comments_df = queryBycategrot(df, i)
        re_str = '[1-9][0-9]* *' + opt
        gr_comments_df['pages'] = gr_comments_df['comments'].map(lambda x: re.findall(re_str, str(x)))
        # data_1 = gr_comments_df.copy()
        # data_1 = data_1[data_1['pages'].apply(len) > 0]
        ownpages_df = gr_comments_df.copy()
        ownpages_df = ownpages_df[ownpages_df['pages'].apply(len) > 0]
        ownpages_df['pages'] = ownpages_df['pages'].apply(lambda x: float(x[0].replace(opt, '')))
        page_data[i] = ownpages_df['pages']
    return page_data.melt()

for i in ['pages', 'tables', 'figures']:
    desc_df = get_datasofcommits(all_df, i)
    desc_df.to_csv(f'../output/{i}-data.csv', index=False)

C:\Users\马\AppData\Local\Temp\ipykernel_15124\10698150.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gr_comments_df['pages'] = gr_comments_df['comments'].map(lambda x: re.findall(re_str, str(x)))
C:\Users\马\AppData\Local\Temp\ipykernel_15124\10698150.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gr_comments_df['pages'] = gr_comments_df['comments'].map(lambda x: re.findall(re_str, str(x)))
C:\Users\马\AppData\Local\Temp\ipykernel_15124\10698150.py:15: SettingWithCopyWarning: 
A value is trying t